In [6]:
import os
current_path = os.getcwd()
print(current_path)

from google.colab import drive
drive.mount('/content/drive')

/content
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import zipfile
DATA_IN_PATH = '/content/drive/MyDrive/DeepLearning_NLP/data_in/'
file_list = ['labeledTrainData.tsv.zip', 'testData.tsv.zip', 'unlabeledTrainData.tsv.zip']

for file in file_list:
  zipRef = zipfile.ZipFile(DATA_IN_PATH + file, 'r')
  zipRef.extractall(DATA_IN_PATH)
  zipRef.close()

In [11]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

print("File Size: ")
for file in os.listdir(DATA_IN_PATH):
  if 'tsv' in file and 'zip' not in file:
    print(file.ljust(30) + str(round(os.path.getsize(DATA_IN_PATH + file) / 1000000, 2)) + 'MB')
train_data = pd.read_csv(DATA_IN_PATH + 'labeledTrainData.tsv', header = 0, delimiter = '\t', quoting = 3)
print('The number of entire trainign data: {}'.format(len(train_data)))
train_length = train_data['review'].apply(len)
train_length.head()

File Size: 
labeledTrainData.tsv          33.56MB
testData.tsv                  32.72MB
unlabeledTrainData.tsv        67.28MB
The number of entire trainign data: 25000


0    2304
1     948
2    2451
3    2247
4    2233
Name: review, dtype: int64

In [12]:
import re 
import json 
import pandas as pd 
import numpy as np 
import nltk 
from bs4 import BeautifulSoup 
from nltk.corpus import stopwords 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer

DATA_IN_PATH = '/content/drive/MyDrive/DeepLearning_NLP/data_in/'
train_data = pd.read_csv(DATA_IN_PATH + 'labeledTrainData.tsv', header = 0, delimiter = '\t', quoting = 3)
review = train_data['review'][0]
review_text = BeautifulSoup(review, 'html5lib').get_text()
review_text = re.sub("[^a-zA-Z]", " ", review_text)
review_text = review_text.lower()
words = review_text.split()
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
words = [w for w in words if not w in stop_words]
print(words)

clean_review = ' '.join(words)
print(clean_review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker', 'maybe', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'really', 'cool', 'eighties', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent', 'moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'going', 'see', 'cinema', 'originally', 'released', 'subtle', 'messages', 'mj', 'feeling', 'towards', 'press', 'also', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'course', 'michael', 'jackson', 'unless', 'remotely', 'like', 'mj', 'anyway', 'going', 'hate', 'find', 'boring', 'may', 'call', 'mj', 'egotist', 'consenting', 'making', 'movie', 'mj', 'fans', 'would', 'say', 'made', 'fans', 'true', 'really', 'nice', 'actual', 'feature', 'film', 'bit', 'finally', 'starts', 'minutes', 'excluding

In [13]:
def preprocessing(review, remove_stopwords = False):
  review_text = BeautifulSoup(review, 'html5lib').get_text()
  review_text = re.sub("[^a-zA-Z]", " ", review_text)
  review_text = review_text.lower()
  if remove_stopwords:
    words = review_text.split()
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]
    review_text = ' '.join(words)
  else :
    review_text = ' '.join(words)
  return review_text
clean_train_reviews = []
for review in train_data['review']:
  clean_train_reviews.append(preprocessing(review, remove_stopwords = True))
clean_train_df = pd.DataFrame({'review': clean_train_reviews, 'sentiment': train_data['sentiment']})

In [15]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_reviews)
text_sequences = tokenizer.texts_to_sequences(clean_train_reviews)

MAX_SEQUENCE_LENGTH = 174
train_inputs = pad_sequences(text_sequences, maxlen = MAX_SEQUENCE_LENGTH, padding = 'post')
print('Train Data:', train_inputs.shape)

train_labels = np.array(train_data['sentiment'])
print('Label:', train_labels.shape)

Train Data: (25000, 174)
Label: (25000,)


In [18]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])
vectorizer = TfidfVectorizer(min_df = 0.0, analyzer='char', sublinear_tf=True, ngram_range=(1,3), max_features = 5000)
X = vectorizer.fit_transform(reviews)
y = np.array(sentiments)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
lgs = LogisticRegression(class_weight = 'balanced')
lgs.fit(X_train, y_train)
print("Accuracy: %f" % lgs.score(X_test, y_test))

(20000, 5000) (5000, 5000) (20000,) (5000,)
Accuracy: 0.869600


In [19]:
from keras.models import Sequential
from keras.layers import Dense
X_train1 = X_train.toarray()
X_test1 = X_test.toarray()

model = Sequential()
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer = 'nadam', metrics = ['accuracy'])
model.fit(X_train1, y_train, epochs=200, verbose = 1)

_, accuracy = model.evaluate(X_test1, y_test)
print('Accuracy : ', accuracy)
model.summary()

Epoch 1/200
625/625 [==============================] - 4s 4ms/step - loss: 0.6410 - accuracy: 0.7582
Epoch 2/200
625/625 [==============================] - 2s 3ms/step - loss: 0.5597 - accuracy: 0.8180
Epoch 3/200
625/625 [==============================] - 2s 4ms/step - loss: 0.5059 - accuracy: 0.8321
Epoch 4/200
625/625 [==============================] - 2s 3ms/step - loss: 0.4677 - accuracy: 0.8433
Epoch 5/200
625/625 [==============================] - 4s 7ms/step - loss: 0.4389 - accuracy: 0.8505
Epoch 6/200
625/625 [==============================] - 2s 4ms/step - loss: 0.4165 - accuracy: 0.8563
Epoch 7/200
625/625 [==============================] - 2s 4ms/step - loss: 0.3985 - accuracy: 0.8609
Epoch 8/200
625/625 [==============================] - 2s 4ms/step - loss: 0.3837 - accuracy: 0.8648
Epoch 9/200
625/625 [==============================] - 2s 4ms/step - loss: 0.3714 - accuracy: 0.8670
Epoch 10/200
625/625 [==============================] - 3s 4ms/step - loss: 0.3608 - accura